In [1]:
script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using JLD
using Plots
using ColorSchemes
using Pkg
Pkg.add("StatsPlots")
using StatsPlots

println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8572467637776589285\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17006312351215922362\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-8572467637776589285\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


importing packages...
  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]
packages and functions imported
Elapsed time = 36.46405220031738 seconds 
 


In [2]:
### Set parameters

# Parameters for all graphs
const NNODES = 70
const MAXDIM = 3    # Maximum persistent homology dimension
const NREPS = 50
const DATESTRING = "082520"


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("bettiBars",x), readdir(read_dir))
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end


# Locate the nametags
nametags = []
for betti_file in betti_files
    tag = split(split(betti_file, "$(DATESTRING)_")[2], "_bettiBars")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)

Located the following graph files:
DP_50_70_3_082520_backward_bettiBars.jld
DP_50_70_3_082520_forward_bettiBars.jld
DP_50_70_3_082520_thresh005_edge121_threshold_bettiBars.jld
DP_50_70_3_082520_thresh005_edge121_threshold_bettiBars_postnoise.jld
DP_50_70_3_082520_thresh005_edge121_threshold_bettiBars_prenoise.jld
DP_50_70_3_082520_thresh015_edge363_threshold_bettiBars.jld
DP_50_70_3_082520_thresh015_edge363_threshold_bettiBars_postnoise.jld
DP_50_70_3_082520_thresh015_edge363_threshold_bettiBars_prenoise.jld
DP_50_70_3_082520_thresh01_edge242_threshold_bettiBars.jld
DP_50_70_3_082520_thresh01_edge242_threshold_bettiBars_postnoise.jld
DP_50_70_3_082520_thresh01_edge242_threshold_bettiBars_prenoise.jld
DP_50_70_3_082520_thresh02_edge483_threshold_bettiBars.jld
DP_50_70_3_082520_thresh02_edge483_threshold_bettiBars_postnoise.jld
DP_50_70_3_082520_thresh02_edge483_threshold_bettiBars_prenoise.jld
DP_50_70_3_082520_thresh03_edge725_threshold_bettiBars.jld
DP_50_70_3_082520_thresh03_edge725_

coreperiph_700_100_100_20_20_09_05_082520_thresh04_edge966_threshold_bettiBars_postnoise.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh04_edge966_threshold_bettiBars_prenoise.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh05_edge1208_threshold_bettiBars.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh05_edge1208_threshold_bettiBars_postnoise.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh05_edge1208_threshold_bettiBars_prenoise.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh06_edge1449_threshold_bettiBars.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh06_edge1449_threshold_bettiBars_postnoise.jld
coreperiph_700_100_100_20_20_09_05_082520_thresh06_edge1449_threshold_bettiBars_prenoise.jld
cosineGeometric_50_70_3_082520_backward_bettiBars.jld
cosineGeometric_50_70_3_082520_forward_bettiBars.jld
cosineGeometric_50_70_3_082520_thresh005_edge121_threshold_bettiBars.jld
cosineGeometric_50_70_3_082520_thresh005_edge121_threshold_bettiBars_postnoise.jld
cosineGeom

10-element Array{Any,1}:
 "backward"                   
 "forward"                    
 "thresh005_edge121_threshold"
 "thresh015_edge363_threshold"
 "thresh01_edge242_threshold" 
 "thresh02_edge483_threshold" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_threshold"
 "thresh06_edge1449_threshold"

In [3]:
for nametag in nametags
    
    ## Create one large plot with all of the betti curves
    betti_files_nametag = filter(x -> occursin("$(nametag)",x), betti_files)
    model_names = [split(betti_file_nametag,"_")[1] for betti_file_nametag in betti_files_nametag]
  

    ### Create one large plot with all of the betti curves
    nModels = length(betti_files_nametag)
    nEdges = binomial(NNODES, 2)
    bettiBarAll = zeros(NREPS, MAXDIM, nModels)
    muBarAll = zeros(NREPS, MAXDIM, nModels)
    nuBarAll = zeros(NREPS, MAXDIM, nModels)
    
    for (i,betti_file_nametag) in enumerate(betti_files_nametag)

        # Read in Betti curves
        bettiBar_dict = load("$(read_dir)/$(betti_file_nametag)")
        bettiBarAll[:,:, i] = bettiBar_dict["bettiBarArray"]
        muBarAll[:,:, i] = bettiBar_dict["muBarArray"]
        nuBarAll[:,:, i] = bettiBar_dict["nuBarArray"]

    end


    println("$(nametag) Size bettiBarAll is $(size(bettiBarAll))")



    # Plot
    p1a = violin(bettiBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p1b = violin(bettiBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p1c = violin(bettiBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("BettiBar")

    plot(p1a, p1b, p1c, layout = (3,1))

    savefig("../figures/all_bettiBars_$(nametag).pdf")


    p2a = violin(muBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p2b = violin(muBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p2c = violin(muBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("muBar")

    plot(p2a, p2b, p2c, layout = (3,1))

    savefig("../figures/all_muBars_$(nametag).pdf")


    p3a = violin(nuBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p3b = violin(nuBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p3c = violin(nuBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("muBar")

    plot(p3a, p3b, p3c, layout = (3,1))

    savefig("../figures/all_nuBars_$(nametag).pdf")

end

backward Size bettiBarAll is (50, 3, 10)
forward Size bettiBarAll is (50, 3, 10)
thresh005_edge121_threshold Size bettiBarAll is (50, 3, 30)
thresh015_edge363_threshold Size bettiBarAll is (50, 3, 30)
thresh01_edge242_threshold Size bettiBarAll is (50, 3, 30)
thresh02_edge483_threshold Size bettiBarAll is (50, 3, 30)
thresh03_edge725_threshold Size bettiBarAll is (50, 3, 30)
thresh04_edge966_threshold Size bettiBarAll is (50, 3, 30)
thresh05_edge1208_threshold Size bettiBarAll is (50, 3, 30)
thresh06_edge1449_threshold Size bettiBarAll is (50, 3, 30)
